In [27]:
import pandas as pd
import datetime

In [112]:
df_users = pd.read_csv("/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4012/Data/combined_twitter_data_with_tweets_corpus.csv")

In [9]:
# get tweets df
base = "/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4012/"
filenames_tweets = [
    "Data/all tweets 2017/tweets_fake_followers.csv",
    "Data/all tweets 2017/tweets_genuine_accounts.csv",
    "Data/all tweets 2017/tweets_social_spambots_1.csv",
    "Data/all tweets 2017/tweets_social_spambots_2.csv",
    "Data/all tweets 2017/tweets_social_spambots_3.csv",
    "Data/all tweets 2017/tweets_traditional_spambots_1.csv",

    "Data/tweets 2015/tweets_E13.csv",
    "Data/tweets 2015/tweets_FSF.csv",
    "Data/tweets 2015/tweets_INT.csv",
    "Data/tweets 2015/tweets_TFP.csv",
    "Data/tweets 2015/tweets_TWT.csv"
]
filenames_tweets = map(lambda x: base+x, filenames_tweets)

for i,fn in enumerate(filenames_tweets):
    if i == 0:
        df_tweets = pd.read_csv(fn, encoding='ISO-8859-1')
    else:
        df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1643796394.py:21: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.read_csv(fn, encoding='ISO-8859-1')
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1643796394.py:23: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1643796394.py:23: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd.read_csv(fn, encoding='ISO-8859-1') ], axis=0)
/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1643796394.py:23: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tweets = pd.concat([df_tweets, pd

In [15]:
df_tweets = df_tweets.dropna(subset = ["user_id"])  
df_tweets["user_id"] = df_tweets["user_id"].apply(int)

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/1418784699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets["user_id"] = df_tweets["user_id"].apply(int)


In [39]:
# takes around 10 min to run
df_tweets['created_at_formatted'] = pd.to_datetime(df_tweets['timestamp'], infer_datetime_format=True, errors='coerce')

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/2158826432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['created_at_formatted'] = pd.to_datetime(df_tweets['timestamp'], infer_datetime_format=True, errors='coerce')


In [105]:
# user tweet frequency = total number of tweets / number of user active days 
# shows how often the user tweets among the days that a user tweets at least once. User activity is defined by whether the user tweets in a given day
# 1 = user tweets only once per active day 
# >1 = user tweets more than once a day on average, in the days that the user is active 

df_tweets_per_day = df_tweets.groupby(by=["user_id"]).agg(tweet_count=('text', 'count'),
                                                          date_count=('created_at_formatted', lambda x: x.nunique()))

dict_tweets_average = {user_id: df_tweets_per_day.loc[user_id]['tweet_count'] / df_tweets_per_day.loc[user_id]['date_count'] for user_id in df_tweets_per_day.index}

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_84439/2468365992.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['tweet_placeholder'] = 1


In [114]:
#create new column for user tweet frequency 
df_users['tweet_frequency'] = df_users['id'].map(dict_tweets_average)

In [116]:
df_tweets.head() 

,created_at,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,...,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,timestamp,crawled_at,updated,created_at_formatted,tweet_placeholder
0,Sat Apr 20 13:19:19 +0000 2013,325599560959393793,https://t.co/iocNIgHxXH. @LovesOfaLDNgirl her...,"<a href=""http://twitter.com/download/iphone"" r...",10935572,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,0.0,1.0,1.0,2013-04-20 15:19:19,NaN,NaN,2013-04-20 15:19:19,1
1,Tue Apr 16 19:31:39 +0000 2013,324243711443730434,Well done hubby @Allan_76 http://t.co/AaeTwLucUG,"<a href=""http://instagram.com"" rel=""nofollow"">...",10935572,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,0.0,1.0,1.0,2013-04-16 21:31:39,NaN,NaN,2013-04-16 21:31:39,1
2,Tue Apr 16 17:38:06 +0000 2013,324215137055670274,Two years with my lovely husband - thank you f...,"<a href=""http://instagram.com"" rel=""nofollow"">...",10935572,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,0.0,1.0,1.0,2013-04-16 19:38:06,NaN,NaN,2013-04-16 19:38:06,1
3,Sun Apr 14 15:33:00 +0000 2013,323458877003792386,Sorry bunny about your ears but I was hungry.....,"<a href=""http://instagram.com"" rel=""nofollow"">...",10935572,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,0.0,1.0,0.0,2013-04-14 17:33:00,NaN,NaN,2013-04-14 17:33:00,1
4,Fri Apr 12 15:37:59 +0000 2013,322735354148945920,"Small man, big drink @Allan_76 http://t.co/4NU...","<a href=""http://instagram.com"" rel=""nofollow"">...",10935572,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,0.0,1.0,1.0,2013-04-12 17:37:59,NaN,NaN,2013-04-12 17:37:59,1


In [ ]:
# return 0 if weekend, 1 if weekday 
def is_weekday(dt):
    return 0 if dt.weekday() > 4 else 1

# return day of week 
def get_weekday(dt):
    return dt.weekday()